In [2]:
from Data_loader import *
from Final_Model import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

New behaviors: 
    user_id                                     candidate_news  \
0    22779  [9774516, 9771051, 9770028, 9775402, 9774461, ...   
1   150224  [9778669, 9778736, 9778623, 9089120, 9778661, ...   
2   160892  [9778369, 9777856, 9778500, 9778021, 9778627, ...   
3  1001055  [9776715, 9776406, 9776566, 9776071, 9776808, ...   
4  1001055  [9775202, 9776855, 9776688, 9771995, 9776583, ...   

  article_ids_clicked                                        clicked_idx  
0           [9759966]                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]  
1           [9778661]  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
2           [9777856]                  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]  
3           [9776566]                        [0, 0, 1, 0, 0, 0, 0, 0, 0]  
4           [9776553]  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
Loading GloVe vectors from saved file: ../Data/glove_vectors.pt


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 5105984000 bytes.

In [ ]:
print("\nbrowsed news: ", browsed_news_train.shape
      , "\ncandidate news: ", candidate_news_train.shape
      , "\nclicked news: ", clicked_news_train.shape)

In [ ]:
### Test data loader (do not run if trying to test the training loop)

batch_size = 16

### Test news encoder
print("\n ------NEWS ENCODER------")

word_embedding_matrix = glove_vectors  # Assume glove_vectors are loaded and of correct size
attention_dim = 200
# Instantiate the model
news_encoder = NewsEncoder(embed_size=300, heads=15, word_embedding_matrix=word_embedding_matrix, attention_dim=attention_dim)

# Random input

x = browsed_news_train[:batch_size, 1, :] #[Batch size, 1 news, 26 words]

output = news_encoder(x)

print("input shape:", x.shape)
print("output shape:", output.shape) # News encoder works fine




### Test user encoder
print('\n ------USER ENCODER------')

user_encoder = UserEncoder(embed_size=300, heads=15, attention_dim=200)

x = browsed_news_train[:batch_size, :, :] #[Batch size, all news, 26 words]

e = [news_encoder(news) for news in x] # Apply the news encoder to each news article
e = torch.stack(e, dim=0)


output = user_encoder(e)

print("input shape:", e.shape)
print("output shape:", output.shape) # User encoder works fine

### Test full model
print('\n -----COMPLETE MODEL------') 

batch_size = 3

model_final = NRMS(embed_size=300, heads=15, word_embedding_matrix=glove_vectors, attention_dim=200)

browsed_news_batch = browsed_news_train[:batch_size, :, :] #[Batch size, all news, 26 words]
candidate_news_batch = candidate_news_train[:batch_size, :, :] #[Batch size, all news, 26 words]
clicked_news_batch = clicked_news_train[:batch_size, :] #[Batch size, all news]

# Forward pass for the entire batch
click = model_final(browsed_news_batch, candidate_news_batch)

print(f"\nInput shape: Browsed = {browsed_news_batch.shape}, Candidate = {candidate_news_batch.shape}")
print(f"Output shape: Click = {click.shape}") # Full model works fine

print("\nSum of probabilities:", torch.sum(click, dim=1)) # Probabilities sum to 1
print(f"\nClicked indices: True = {torch.argmax(click, dim=1)}, Predicted = {torch.argmax(clicked_news_batch, dim=1)}") # Predicted indices match true indices


 ------NEWS ENCODER------


NameError: name 'glove_vectors' is not defined

In [ ]:
#initialize the model
model_final = NRMS(embed_size=300, heads=15, word_embedding_matrix=glove_vectors, attention_dim=200)

# Define loss function and optimizer
optimizer = torch.optim.Adam(model_final.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    total_loss = 0
    

    for i in range(0, len(browsed_news_train), batch_size):
        browsed_news_batch = browsed_news_train[i:i+batch_size, :, :]
        candidate_news_batch = candidate_news_train[i:i+batch_size, :, :]
        clicked_news_batch = clicked_news_train[i:i+batch_size, :]

        # Forward pass
        click = model_final(browsed_news_batch, candidate_news_batch)


        # Compute loss
        criterion = nn.CrossEntropyLoss()
        loss = criterion(click, torch.argmax(clicked_news_batch, dim=1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(browsed_news_train)}")
